## CIFAR 이미지 분류 모델 구현

- 데이터 : https://www.cs.toronto.edu/~kriz/cifar.html
- 학습방법 : 10가지 이미지 종류 분류 => 다중분류
- 이미지 : DNN(MLP), CNN 적용

### [1] 데이터 로딩

In [1]:
from keras.datasets.cifar10 import load_data

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
from tensorflow.keras.utils import set_random_seed, plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.datasets.fashion_mnist import load_data
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# W, B 고정
set_random_seed(11)

In [5]:
print(f'훈련 입력 데이터 : {x_train.shape}', f'훈련 타겟 데이터 : {y_train.shape}')
print(f'테스트 입력 데이터 : {y_test.shape}', f'테스트 타겟 데이터 : {y_test.shape}')

훈련 입력 데이터 : (50000, 32, 32, 3) 훈련 타겟 데이터 : (50000, 1)
테스트 입력 데이터 : (10000, 1) 테스트 타겟 데이터 : (10000, 1)


### [2] 데이터 전처리 및 학습형태로 변환

In [6]:
# 스케일링
x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
# 정규화
#train과 test셋 모두 정규화 작업    

mean=[0,0,0]
std=[0,0,0]

newx_train = np.ones(x_train.shape)
newx_test = np.ones(x_test.shape)

#train set에 있는 데이터로만 평균과 표준편차를 구함
for i in range(3):
    mean[i] = np.mean(x_train[:,:,:,i])
    std[i] = np.std(x_train[:,:,:,i])


for i in range(3):
    newx_train[:,:,:,i] = x_train[:,:,:,i] - mean[i]
    newx_train[:,:,:,i] = newx_train[:,:,:,i] / std[i]
    newx_test[:,:,:,i] = x_test[:,:,:,i] - mean[i]
    newx_test[:,:,:,i] = newx_test[:,:,:,i] / std[i]
        
x_train = newx_train
x_test = newx_test

In [8]:
# Conv 레이어는 이미지의 채널 정보까지 입력
x_train= x_train.reshape(-1, 32, 32, 3)
x_test= x_test.reshape(-1, 32, 32, 3)

In [9]:
print(f'훈련 입력 데이터 : {x_train.shape}')
print(f'테스트 입력 데이터 : {x_test.shape}')

훈련 입력 데이터 : (50000, 32, 32, 3)
테스트 입력 데이터 : (10000, 32, 32, 3)


### [3] 모델 구성 및 생성
- 입력 형태 : 채널 정보까지 포함 3차원 (32, 32, 3)
- 출력 형태 : 0 ~ 9까지 정수 확률값 10개 출력
- 학습 방식 : 분류 중에서 다중분류
- 전반부
    * 이미지 특징 추출
        - Con2D, MaxPoll2D
- 후반부
    * 이미지 데이터 학습
        - Flatten, Dense

#### [3] - 1 모델 구상

In [10]:
model = Sequential()

# 이미지 특징 추출 Layer => 첫번째 입력값 설정
model.add(Conv2D(3, kernel_size = 3, padding = 'same', input_shape = (32, 32, 3)))

# 이미지 특징 다운샘플링 해주는 Layer => MaxPoll2D
model.add(MaxPool2D(pool_size = (2,2)))

# 이미지를 1차원으로 변환시켜주는 Layer
model.add(Flatten())

# 출력층 => Node : 10(0-9), activation : softmax
model.add(Dense(10, activation = 'softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 3)         84        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 3)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 10)                7690      
                                                                 
Total params: 7,774
Trainable params: 7,774
Non-trainable params: 0
_________________________________________________________________


#### [3] - 2 모델 생성
---  
- complie 메서드
    * loss 손실 함수 : categorical_crossentropy
    * optimizer 최적화 : adam
    * metrics 평가항목 : acuuracy

In [12]:
model.compile(loss = 'sparse_categorical_crossentropy',
optimizer='adam',
metrics='accuracy')

#### [3] - 3 모델 학습
- fit 메서드
    - 학습 데이터, 라벨
    - epochs : 학습 횟수
    - batch_size : 학습 분량
    - validation_data : 검증 데이터, 검증 라벨
    - validataion_split 학습데이터의 일부 비율 설정(ex, 0.2)
    - callback
    - verbose : 학습 진행도 화면 출력 여부 설정

In [13]:
model.fit(x_train, y_train, epochs = 30, batch_size = 30)

Epoch 1/100
1667/1667 [==============================] - 11s 6ms/step - loss: 1.6835 - accuracy: 0.4049
Epoch 2/100
1667/1667 [==============================] - 10s 6ms/step - loss: 1.5348 - accuracy: 0.4599
Epoch 3/100
1667/1667 [==============================] - 11s 7ms/step - loss: 1.4811 - accuracy: 0.4806
Epoch 4/100
1667/1667 [==============================] - 13s 8ms/step - loss: 1.4427 - accuracy: 0.4952
Epoch 5/100
1667/1667 [==============================] - 16s 10ms/step - loss: 1.4193 - accuracy: 0.5025
Epoch 6/100
1667/1667 [==============================] - 15s 9ms/step - loss: 1.4012 - accuracy: 0.5108
Epoch 7/100
1667/1667 [==============================] - 15s 9ms/step - loss: 1.3877 - accuracy: 0.5141
Epoch 8/100
1667/1667 [==============================] - 14s 9ms/step - loss: 1.3808 - accuracy: 0.5154
Epoch 9/100
1667/1667 [==============================] - 14s 8ms/step - loss: 1.3725 - accuracy: 0.5210
Epoch 10/100
1667/1667 [==============================] - 11s 7

KeyboardInterrupt: 

### [4] 평가

In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.4454 - accuracy: 0.4908


[1.4453754425048828, 0.49079999327659607]